# Rescore

In [ ]:
from dotenv import load_dotenv
import os
from time import sleep, time
import json
from openai import OpenAI
import requests
from tqdm import tqdm
import matplotlib.pyplot as plt

import pandas as pd
load_dotenv()

In [ ]:
origindf = pd.read_excel('score.xlsx')

In [ ]:
class DataExtraction:
  def __init__(self, df:pd.DataFrame) -> None:
    self._model = os.getenv("LLM_MODEL")
    self._seed = os.getenv("SEED")
    self._df = df.copy()
    self._df['llm'] = self._df[self._model]
  
  def get_df(self):
    return self._df

  def get_row_by_index(self, i:int):
    return self._df.iloc[i]
    
  def get_url_by_index(self, i:int):
    row = self._df.iloc[i]
    return {
      'citing':f'https://www.semanticscholar.org/paper/{row.paper_id}',
      'cited':f'https://www.semanticscholar.org/paper/{row.cited_id}'
    }

  def high_cross_high_llm(self):
    df = self._df
    cand = df[(df['relevance_cross']>=0.95) & (df["llm"] >= 0.95)]
    return cand[['relevance_cross','llm']]
  def high_cross_mid_llm(self):
    df = self._df
    cand = df[(df['relevance_cross']>=0.8) & (df["llm"] >= 0.40) & (df["llm"] <= 0.6)]
    return cand[['relevance_cross','llm']]
  def high_cross_low_llm(self):
    df = self._df
    cand = df[(df['relevance_cross']>=0.8) & (df["llm"] <= 0.2)]
    return cand[['relevance_cross','llm']]

  def low_cross_low_llm(self):
    df = self._df
    cand = df[(df['relevance_cross']<=0.1) & (df["llm"] <= 0.1)]
    return cand[['relevance_cross','llm']]
  def low_cross_mid_llm(self):
    df = self._df
    cand = df[(df['relevance_cross']<=0.2) & (df["llm"] >= 0.40) & (df["llm"] <= 0.6)]
    return cand[['relevance_cross','llm']]
  def low_cross_high_llm(self):
    df = self._df
    cand = df[(df['relevance_cross']<=0.2) & (df["llm"] >= 0.9)]
    return cand[['relevance_cross','llm']]

  def select_case(self, cross_cond, llm_target, tol=0.05):
    """
    从 df 中选择一个样本：
    - cross_cond: lambda x: bool
    - llm_target: 目标 llm 值（0.8 / 0.5 / 0.2）
    - tol: 允许误差
    """
    df = self._df
    cand = df[df["relevance_cross"].apply(cross_cond)]
    cand = cand[(cand["llm"] >= llm_target - tol) & (cand["llm"] <= llm_target + tol)]
    return cand.sample(1).iloc[0] if len(cand) > 0 else None



In [ ]:
data_instance = DataExtraction(origindf)
data_instance.high_cross_high_llm()

In [ ]:
data_instance.get_url_by_index(109)

In [ ]:
class LLMRescorer:
    """
    LLM-based rescore module for DAO arbitration.

    L_ij^re = F_LLM(P_i_full, P_j_full)

    This scorer is used only in rescore / arbitration,
    not in routine citation scoring.
    """

    def __init__(self):
        self.client = OpenAI(
            api_key=os.getenv("LLM_KEY"),
            base_url=os.getenv("LLM_URL")
        )
        self.model = os.getenv("LLM_MODEL")

    def rescore(self, contexts:str, citing_text: str, cited_text: str) -> dict:
        """
        Perform full-paper semantic rescore based on two papers' full texts.

        Inputs:
        - citing_text: extracted text of the citing paper (PDF -> text)
        - cited_text: extracted text of the cited paper (PDF -> text)

        Returns:
        Structured JSON with score and explanations.
        """
        
        prompt = f"""
You are an expert academic reviewer participating in a post-publication
citation arbitration process.

Your task is to reassess the semantic relevance of a specific citation
between two papers.

The goal is NOT to evaluate overall paper similarity, nor to judge whether
the cited paper is central to the entire citing paper.
Instead, you must evaluate whether THIS PARTICULAR CITATION is
semantically justified and substantively supported.

The citation context below indicates WHERE and HOW the cited paper
is referenced in the citing paper.
Use this context as an anchor to locate the relevant discussion
within the full text of the citing paper.

You may consult the full text of both papers to determine whether
the citation is meaningfully supported beyond the local context,
including methodological usage, theoretical grounding,
empirical comparison, or conceptual dependency.

Ignore citation counts, venue prestige, and author identities.
Focus strictly on semantic justification of this citation.

---

Citation context (attention anchor):

\r\n
{contexts}
\r\n

Citing paper (full text):

\r\n
{citing_text}
\r\n

Cited paper (full text):

\r\n
{cited_text}
\r\n

---

Return your assessment strictly in the following JSON format:

{{
  "score": 0.0,
  "methodology": "Whether the cited paper provides methods, algorithms, or tools used in this citation.",
  "theory": "Whether the cited paper contributes theoretical or conceptual foundations relevant to this citation.",
  "evidence": "Whether the cited paper supplies empirical evidence, benchmarks, or comparisons relevant to this citation.",
  "dependency": "Overall assessment of how substantively this citation depends on the cited paper.",
  "summary": "One-sentence justification of the final relevance judgment."
}}

The score must be a real number between 0 and 1, where:
- 0 indicates a weak, perfunctory, or nominal citation, and
- 1 indicates a strongly justified and semantically substantial citation.

Each field must contain a concise and concrete explanation.
Do NOT include any text outside the JSON object.

"""

        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {
                    "role": "system",
                    "content": "You are a careful, neutral, and consistent academic reviewer."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            response_format={"type": "json_object"},
            # temperature=0.0  # arbitration 必须可复现
        )
        try:
            result = json.loads(response.choices[0].message.content)
        except Exception as e:
            result = {
                'score': 0,
                'methodology':'',
                'theory':'',
                'evidence':'',
                'dependency':'',
                'summary': str(e)
            }

        # safety clamp
        result["score"] = float(
            min(max(result.get("score", 0.0), 0.0), 1.0)
        )

        return result


In [ ]:
class RescoreProsssor:

  def __init__(self, obj, citing:str, cited:str):
    self._scorer = LLMRescorer()
    self._df = pd.DataFrame([{
      'score': obj.llm,
      'contexts': obj.contexts,
      'abstract': obj.cited_abstract,
      'methodology': obj.methodology,
      'theory': obj.theory,
      'evidence': obj.evidence,
      'dependency': obj.dependency,
      'summary': obj.summary
    }])
    self._contexts = obj.contexts
    self._citing = self.pdf_to_text_grobid(citing)
    self._cited = self.pdf_to_text_grobid(cited)
  
  def get_df(self):
    return self._df

  def pdf_to_text_grobid(self, pdf_path):
    with open(pdf_path, "rb") as f:
        r = requests.post(
            "http://localhost:8070/api/processFulltextDocument",
            files={"input": f}
        )
    return r.text  # TEI XML

  def repeat(self, filename:str, n = 10):
    with tqdm(total=n, desc="处理中") as pbar:
      while n > 0:
        res = self._scorer.rescore(self._contexts, self._citing, self._cited)
        self._df = pd.concat([self._df, pd.DataFrame([res])], ignore_index=True)
        self._df.to_excel(filename)
        pbar.set_postfix({"n": n, 'score':res.get('score', 0)})
        pbar.update()  # 更新1个单位
        n -= 1
    pbar.close()
    return self._df

## resocre

- high_cross_high_llm: 551
- high_cross_mid_llm: 21
- high_cross_low_llm: 701

- low_cross_low_llm: 472
- low_cross_mid_llm: 640
- low_cross_high_llm: 109

In [ ]:
index = 109
data = data_instance.get_row_by_index(index)
instance = RescoreProsssor(data, f'rescore/{index}citing.pdf', f'rescore/{index}cited.pdf')
instance.repeat(f'rescore/{index}.xlsx')

In [ ]:

class ScoreShower:
  def __init__(self):
    self._cols = {
      'high_cross_high_llm': 551,
      'high_cross_mid_llm': 21,
      'high_cross_low_llm': 701,

      'low_cross_low_llm': 472,
      'low_cross_mid_llm': 640,
      'low_cross_high_llm': 109
    }
    self._df = pd.DataFrame()
  def plot(self):
    for k, v in self._cols.items():
      df = pd.read_excel(f'rescore/{v}.xlsx')
      self._df[k] = df['score'].values
    fig, ax = plt.subplots(figsize=(7.2, 5))
    self._df.plot(ax=ax)
    ax.set_xlabel(r"$t$")
    ax.set_ylabel("Score")
    return self._df
ss = ScoreShower()
ss.plot()